#Домашняя работа № 3. Гибридные рекомендательные системы

##Набор данных MovieLens объемом 100 тыс.

Посмотрим на оригинальный датасет

In [2]:
import pandas as pd
import numpy as np

In [3]:
# загрузим данные
!wget 'https://files.grouplens.org/datasets/movielens/ml-100k.zip'

--2025-02-02 10:55:05--  https://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  8.09MB/s    in 0.6s    

2025-02-02 10:55:06 (8.09 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]



In [4]:
# распакуем
import zipfile

with zipfile.ZipFile('/content/ml-100k.zip', 'r') as zip_ref:
    zip_ref.extractall('unpacked_files')

In [5]:
# прочитаем описание README
"""
ПОДРОБНЫЕ ОПИСАНИЯ ФАЙЛОВ ДАННЫХ
==============================================

Вот краткое описание данных.

ml-data.tar.gz -- Сжатый файл tar.  Чтобы восстановить файлы данных u, выполните следующее:
                gunzip ml-data.tar.gz
                tar xvf ml-data.tar
                mku.sh

u.data - Полный набор u-данных, 100 000 оценок 943 пользователями по 1682 фильмам.
         Каждый пользователь поставил оценку как минимум 20 фильмам. Пользователи и материалы
         нумеруются последовательно, начиная с 1.  Данные получены случайным образом.
         Это разделенный табуляцией
         список идентификаторов пользователей | элементов | оценок | временных меток.
         Временные метки указаны в секундах с 1.01.1970 UTC в системе unix

u.info - Количество пользователей, элементов и оценок в наборе данных u.

u.item - Информация о фильмах, которые будут показаны в этом разделе;
         список фильмов, которые будут показаны в виде вкладок:
         идентификатор фильма | название фильма | дата выхода | дата выхода видео |
         URL на IMDb | неизвестен | Боевик | Приключения | Анимация |
         Детский | Комедийный | Криминальный | Документальный | Драма | фэнтези |
         Фильм-Нуар | Ужасы | Мюзикл | Мистика | Мелодрама | Научная фантастика |
         Триллер | Военный | Вестерн |
         Последние 19 полей обозначают жанры, 1 указывает на фильм относится к этому жанру,
         0 означает, что это не так; фильмы могут быть в сразу в нескольких жанрах.
         Идентификаторы фильмов используются в наборе данных u.data.

u.genre - список жанров.

u.user - Демографическая информация о пользователях; это вкладка разделенный список
         идентификатор пользователя | возраст | пол | род занятий | почтовый индекс.
         Идентификаторы пользователей используются в наборе данных u.data.

u.occupation - Список профессий.

u1.base - Наборы данных u1.base и u1.test через u5.base и u5.test
u1.test представляют собой 80% /20% разбиений данных u на обучающие и тестовые данные.
u2.base Каждый из наборов тестов u1, ..., u5 имеет непересекающиеся наборы тестов;
        это если для
u2.test 5-кратной перекрестной проверки (где вы повторяете свой эксперимент
        u3.base для каждого обучающего и тестового набора и усредняете результаты).
u3.test Эти наборы данных могут быть сгенерированы из u.data с помощью mku.sh.
u4.base
u4.test
u5.base
u5.test

ua.base - Наборы данных ua.base, ua.test, ub.base и ub.test
ua.test разделяют данные u на обучающий набор и тестовый набор с
ub.base ровно по 10 оценок для каждого пользователя в тестовом наборе.  Наборы
ub.test ua.test и ub.test не пересекаются.  Эти наборы данных могут
        быть сгенерированы из u.data с помощью mku.sh.

allbut.pl -- Скрипта, который генерирует обучающие и тестовые наборы, где
             все оценки пользователей, кроме n, содержатся в обучающих данных.

mku.sh -- Командный скрипт для генерации всех u наборов данных из u.data.

"""

'\nПОДРОБНЫЕ ОПИСАНИЯ ФАЙЛОВ ДАННЫХ\n==============================================\n\nВот краткое описание данных.\n\nml-data.tar.gz -- Сжатый файл tar.  Чтобы восстановить файлы данных u, выполните следующее:\n                gunzip ml-data.tar.gz\n                tar xvf ml-data.tar\n                mku.sh\n\nu.data - Полный набор u-данных, 100 000 оценок 943 пользователями по 1682 фильмам.\n         Каждый пользователь поставил оценку как минимум 20 фильмам. Пользователи и материалы\n         нумеруются последовательно, начиная с 1.  Данные получены случайным образом.\n         Это разделенный табуляцией \n         список идентификаторов пользователей | элементов | оценок | временных меток. \n         Временные метки указаны в секундах с 1.01.1970 UTC в системе unix   \n\nu.info - Количество пользователей, элементов и оценок в наборе данных u.\n\nu.item - Информация о фильмах, которые будут показаны в этом разделе;\n         список фильмов, которые будут показаны в виде вкладок: \n

In [6]:
# прочитаем
u_data = pd.read_csv('/content/unpacked_files/ml-100k/u.data', delimiter='\t')
u_data.columns = ['userID', 'movieID', 'rating', 'time']
u_info = pd.read_csv('/content/unpacked_files/ml-100k/u.info')

# так читать не хочет
# u_item = pd.read_csv('/content/unpacked_files/ml-100k/u.item')
# попробуем по-другому
import csv
csv_file = open('/content/unpacked_files/ml-100k/u.item', newline='', encoding='latin-1')
u_item = pd.read_csv(csv_file, delimiter='|')
u_item.columns = ['movieID', 'movie_title', 'release_date', 'video_release',
                  'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation',
                  'Children_s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
                  'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                  'Thriller', 'War', 'Western']
u_genre = pd.read_csv('/content/unpacked_files/ml-100k/u.genre', delimiter='|')
u_genre.columns = ['genre', '#']
u_user = pd.read_csv('/content/unpacked_files/ml-100k/u.user', delimiter='|')
u_user.columns = ['userID', 'age', 'gender', 'occupation', 'p_code']
u_occupation = pd.read_csv('/content/unpacked_files/ml-100k/u.occupation')
u_occupation.columns = ['occupation']

In [7]:
# пользователь | элемент | рейтинг | временная метка
u_data.head()

,userID,movieID,rating,time
0,186,302,3,891717742
1,22,377,1,878887116
2,244,51,2,880606923
3,166,346,1,886397596
4,298,474,4,884182806


In [8]:
u_info.head()

,943 users
0,1682 items
1,100000 ratings


In [9]:
# идентификатор фильма | название фильма | дата выхода | дата выхода видео |
# URL на IMDb | неизвестен | Боевик | Приключения | Анимация |
# Детский | Комедийный | Криминальный | Документальный | Драма | фэнтези |
# Фильм-Нуар | Ужасы | Мюзикл | Мистика | Мелодрама | Научная фантастика |
# Триллер | Военный | Вестерн |
u_item.head()

,movieID,movie_title,release_date,video_release,IMDb_URL,unknown,Action,Adventure,Animation,Children_s,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,NaN,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
u_item_2 = u_item.copy()
u_item_2.head()

,movieID,movie_title,release_date,video_release,IMDb_URL,unknown,Action,Adventure,Animation,Children_s,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,NaN,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
def genre_to_string (str, num):
  if num == 1:
    string = str

  return

In [82]:
 # добавим единую колонку жанров
 cols = ['Action', 'Adventure', 'Animation', 'Children_s', 'Comedy',
         'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
         'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

for index, row in u_item_2.iterrows():
    string = str()
    for ele in cols:
        if row[ele] == 1:
            if string == '':
                string += ele
            else:
                string += ('|'+ele)
        else:
            continue
    u_item_2.loc[index, 'genre'] = string

In [83]:
u_item_2.head()

,movieID,movie_title,release_date,video_release,IMDb_URL,unknown,Action,Adventure,Animation,Children_s,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre
0,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,Action|Adventure|Thriller
1,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Thriller
2,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Action|Comedy|Drama
3,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Crime|Drama|Thriller
4,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,NaN,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Drama


In [14]:
# посмотрим на список жанров
u_genre

,genre,#
0,Action,1
1,Adventure,2
2,Animation,3
3,Children's,4
4,Comedy,5
5,Crime,6
6,Documentary,7
7,Drama,8
8,Fantasy,9
9,Film-Noir,10


In [15]:
# пользователь | возраст | пол | род занятий | почтовый индекс
u_user.head()

,userID,age,gender,occupation,p_code
0,2,53,F,other,94043
1,3,23,M,writer,32067
2,4,24,M,technician,43537
3,5,33,F,other,15213
4,6,42,M,executive,98101


In [16]:
# профессии
u_occupation.head()

,occupation
0,artist
1,doctor
2,educator
3,engineer
4,entertainment


##LightFM (default)

In [17]:
import numpy as np

In [18]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp311-cp311-linux_x86_64.whl size=831163 sha256=c680fd7a394af13802fe2836f2a09fef71d8d868af3e3223c9e3fc92ae47ba3b
  Stored in directory: /root/.cache/pip/wheels/b9/0d/8a/0729d2e6e3ca2a898ba55201f905da7db3f838a33df5b3fcdd
Successfully built lightfm


In [19]:
from lightfm.datasets import fetch_movielens

movielens = fetch_movielens()

In [20]:
for key, value in movielens.items():
    print(key, type(value), value.shape)

train <class 'scipy.sparse._coo.coo_matrix'> (943, 1682)
test <class 'scipy.sparse._coo.coo_matrix'> (943, 1682)
item_features <class 'scipy.sparse._csr.csr_matrix'> (1682, 1682)
item_feature_labels <class 'numpy.ndarray'> (1682,)
item_labels <class 'numpy.ndarray'> (1682,)


In [21]:
train = movielens['train']
test = movielens['test']

Посмотрим на все это

In [22]:
train.toarray(), len(train.toarray())

(array([[5, 3, 4, ..., 0, 0, 0],
        [4, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [5, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 5, 0, ..., 0, 0, 0]], dtype=int32),
 943)

In [23]:
test.toarray(), len(test.toarray())

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32),
 943)

In [24]:
data = fetch_movielens()
print(repr(data['train']))
print(repr(data['test']))

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 90570 stored elements in COOrdinate format>
<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 9430 stored elements in COOrdinate format>


In [25]:
# зададим рейтинг
data = fetch_movielens(min_rating=5.0)
print(repr(data['train']))
print(repr(data['test']))

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 19048 stored elements in COOrdinate format>
<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 2153 stored elements in COOrdinate format>


In [26]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

model = LightFM()
model.fit(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.44, test 0.13.


In [27]:
model = LightFM(learning_rate=0.05, loss='warp')

model.fit_partial(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.60, test 0.22.


In [28]:
def sample_recommendation(model, data, user_ids):
    n_users, n_items = data['train'].shape
    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()
                          [user_id].indices]

        scores = model.predict(user_id, np.arange(n_items))

        top_items = data['item_labels'][np.argsort(-scores)]

        print("User %s" % user_id)
        print("     Known positives:")

        for x in known_positives[:3]:
            print("        %s" % x)

        print("     Recommended:")

        for x in top_items[:3]:
            print("        %s" % x)

In [29]:
sample_recommendation(model, movielens, [10, 25, 451])

User 10
     Known positives:
        B
        D
        S
     Recommended:
        R
        S
        E
User 25
     Known positives:
        T
        T
        D
     Recommended:
        S
        L
        C
User 451
     Known positives:
        T
        B
        P
     Recommended:
        R
        A
        S


##Соберем свой датафрейм

In [92]:
# объединим u_data и u_user
df_1 = u_data.merge(u_user, on='userID').reset_index(drop=True)
df_1

,userID,movieID,rating,time,age,gender,occupation,p_code
0,186,302,3,891717742,39,F,executive,00000
1,22,377,1,878887116,25,M,writer,40206
2,244,51,2,880606923,28,M,technician,80525
3,166,346,1,886397596,47,M,educator,55113
4,298,474,4,884182806,44,M,executive,01581
...,...,...,...,...,...,...,...,...
99722,880,476,3,880175444,13,M,student,83702
99723,716,204,5,879795543,36,F,administrator,44265
99724,276,1090,1,874795795,21,M,student,95064
99725,13,225,2,882399156,47,M,educator,29206


In [93]:
# объединим df_1 и u_item_2
df_2 = df_1.merge(u_item_2, on='movieID').reset_index(drop=True)
df_2.head()

,userID,movieID,rating,time,age,gender,occupation,p_code,movie_title,release_date,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre
0,186,302,3,891717742,39,F,executive,00000,L.A. Confidential (1997),01-Jan-1997,...,1,0,0,1,0,0,1,0,0,Crime|Film-Noir|Mystery|Thriller
1,22,377,1,878887116,25,M,writer,40206,Heavyweights (1994),01-Jan-1994,...,0,0,0,0,0,0,0,0,0,Children_s|Comedy
2,244,51,2,880606923,28,M,technician,80525,Legends of the Fall (1994),01-Jan-1994,...,0,0,0,0,1,0,0,1,1,Drama|Romance|War|Western
3,166,346,1,886397596,47,M,educator,55113,Jackie Brown (1997),01-Jan-1997,...,0,0,0,0,0,0,0,0,0,Crime|Drama
4,298,474,4,884182806,44,M,executive,01581,Dr. Strangelove or: How I Learned to Stop Worr...,01-Jan-1963,...,0,0,0,0,0,1,0,1,0,Sci-Fi|War


In [94]:
# выберем самое интересное
cols = ['userID', 'movieID', 'movie_title', 'rating', 'age', 'gender', 'occupation',
        'unknown', 'Action', 'Adventure', 'Animation',
        'Children_s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
        'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
        'Thriller', 'War', 'Western', 'genre']
df_data = df_2[cols]
df_data.head()

,userID,movieID,movie_title,rating,age,gender,occupation,unknown,Action,Adventure,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre
0,186,302,L.A. Confidential (1997),3,39,F,executive,0,0,0,...,1,0,0,1,0,0,1,0,0,Crime|Film-Noir|Mystery|Thriller
1,22,377,Heavyweights (1994),1,25,M,writer,0,0,0,...,0,0,0,0,0,0,0,0,0,Children_s|Comedy
2,244,51,Legends of the Fall (1994),2,28,M,technician,0,0,0,...,0,0,0,0,1,0,0,1,1,Drama|Romance|War|Western
3,166,346,Jackie Brown (1997),1,47,M,educator,0,0,0,...,0,0,0,0,0,0,0,0,0,Crime|Drama
4,298,474,Dr. Strangelove or: How I Learned to Stop Worr...,4,44,M,executive,0,0,0,...,0,0,0,0,0,1,0,1,0,Sci-Fi|War


In [95]:
# проверим жанр unknown
df_data['unknown'].sum()

9

In [96]:
# посмотрим на эти строки
df_data.loc[df_data['unknown'] == 1]

,userID,movieID,movie_title,rating,age,gender,occupation,unknown,Action,Adventure,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre
2144,130,267,unknown,5,20,M,none,1,0,0,...,0,0,0,0,0,0,0,0,0,
3732,5,267,unknown,4,33,F,other,1,0,0,...,0,0,0,0,0,0,0,0,0,
7151,268,267,unknown,3,24,M,engineer,1,0,0,...,0,0,0,0,0,0,0,0,0,
8465,181,1373,Good Morning (1971),1,26,M,executive,1,0,0,...,0,0,0,0,0,0,0,0,0,
12332,297,267,unknown,3,29,F,educator,1,0,0,...,0,0,0,0,0,0,0,0,0,
14595,319,267,unknown,4,38,M,programmer,1,0,0,...,0,0,0,0,0,0,0,0,0,
48878,532,267,unknown,3,20,M,student,1,0,0,...,0,0,0,0,0,0,0,0,0,
92840,833,267,unknown,1,34,M,writer,1,0,0,...,0,0,0,0,0,0,0,0,0,
99000,422,267,unknown,4,26,M,entertainment,1,0,0,...,0,0,0,0,0,0,0,0,0,


In [97]:
# считаю целесообразным удалить эти строки
df_data.drop(df_data.index[df_data['movie_title'] == 'unknown'], inplace=True)

<ipython-input-97-9f91ac534a04>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data.drop(df_data.index[df_data['movie_title'] == 'unknown'], inplace=True)


In [98]:
# фильм без жанра тоже удалим
df_data.drop(df_data.index[df_data['movieID'] == 1373], inplace=True)


<ipython-input-98-2853446bdbb8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data.drop(df_data.index[df_data['movieID'] == 1373], inplace=True)


In [99]:
# проверим
df_data.loc[df_data['unknown'] == 1]

,userID,movieID,movie_title,rating,age,gender,occupation,unknown,Action,Adventure,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre


In [106]:
# соберем список уникальных профессий
list_occupations_unique = list(df_data['occupation'].drop_duplicates())
# посмотрим
list_occupations_unique,  len(list_occupations_unique)

(['executive',
  'writer',
  'technician',
  'educator',
  'engineer',
  'librarian',
  'programmer',
  'administrator',
  'student',
  'retired',
  'other',
  'doctor',
  'marketing',
  'artist',
  'lawyer',
  'salesman',
  'homemaker',
  'healthcare',
  'none',
  'entertainment',
  'scientist'],
 21)

In [107]:
#соберем объект жанров по фильмам
series_genre_movies = u_item_2['genre'].str.split('|')
# посмотрим
series_genre_movies[0], series_genre_movies, len(series_genre_movies)

(['Action', 'Adventure', 'Thriller'],
 0       [Action, Adventure, Thriller]
 1                          [Thriller]
 2             [Action, Comedy, Drama]
 3            [Crime, Drama, Thriller]
 4                             [Drama]
                     ...              
 1676                          [Drama]
 1677              [Romance, Thriller]
 1678                 [Drama, Romance]
 1679                         [Comedy]
 1680                          [Drama]
 Name: genre, Length: 1681, dtype: object,
 1681)

##Более интересная модель

In [102]:
from lightfm.data import Dataset

In [110]:
# соберем список уникальных жанров
list_genre_unique = u_genre['genre'].tolist()
list_genre_unique[:5], len(list_genre_unique)

(['Action', 'Adventure', 'Animation', "Children's", 'Comedy'], 18)

In [112]:
# соберем список уникальных профессий
list_occupations_unique = u_occupation['occupation'].tolist()
list_occupations_unique[:5], len(list_occupations_unique)

(['artist', 'doctor', 'educator', 'engineer', 'entertainment'], 20)

In [116]:
# соберем датасет
dataset = Dataset()
dataset.fit(users=df_data['userID'],
            items=df_data['movieID'],
            item_features=list_genre_unique,
            user_features=list_occupations_unique)

In [114]:
# соберем список уникальных пользователей и их профессии
df_unique_userID = df_data.drop_duplicates('userID')
list_user_features = [(x,[y]) for x,y in zip(df_unique_userID['userID'],
                      df_unique_userID['occupation'])
]

list_user_features[:10], len(list_user_features)

([(186, ['executive']),
  (22, ['writer']),
  (244, ['technician']),
  (166, ['educator']),
  (298, ['executive']),
  (115, ['engineer']),
  (253, ['librarian']),
  (305, ['programmer']),
  (6, ['executive']),
  (62, ['administrator'])],
 942)

In [118]:
# соберем список уникальных фильмов и их жанры
df_unique_movieID = df_data.drop_duplicates('movieID')
list_item_features = [(x,y) for x,y in zip(df_unique_movieID['movieID'], series_genre_movies)]

list_item_features[:10], len(list_item_features)

([(302, ['Action', 'Adventure', 'Thriller']),
  (377, ['Thriller']),
  (51, ['Action', 'Comedy', 'Drama']),
  (346, ['Crime', 'Drama', 'Thriller']),
  (474, ['Drama']),
  (265, ['Drama', 'Sci-Fi']),
  (465, ['Children_s', 'Comedy', 'Drama']),
  (451, ['Drama']),
  (86, ['Drama', 'War']),
  (257, ['Crime', 'Thriller'])],
 1679)

In [173]:
df_data.loc[df_data['movieID'] == 111]

,userID,movieID,movie_title,rating,age,gender,occupation,Action,Adventure,Animation,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
86,11,111,"Truth About Cats & Dogs, The (1996)",4,39,F,other,0,0,0,...,0,0,0,0,0,1,0,0,0,0
152,299,111,"Truth About Cats & Dogs, The (1996)",3,29,M,doctor,0,0,0,...,0,0,0,0,0,1,0,0,0,0
456,99,111,"Truth About Cats & Dogs, The (1996)",1,20,M,student,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1348,152,111,"Truth About Cats & Dogs, The (1996)",5,33,F,educator,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1785,62,111,"Truth About Cats & Dogs, The (1996)",3,27,F,administrator,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97354,870,111,"Truth About Cats & Dogs, The (1996)",3,22,M,student,0,0,0,...,0,0,0,0,0,1,0,0,0,0
97942,904,111,"Truth About Cats & Dogs, The (1996)",4,17,F,student,0,0,0,...,0,0,0,0,0,1,0,0,0,0
98113,178,111,"Truth About Cats & Dogs, The (1996)",4,26,M,other,0,0,0,...,0,0,0,0,0,1,0,0,0,0
98868,854,111,"Truth About Cats & Dogs, The (1996)",3,29,F,student,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [ ]:
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)
movies_with_ratings.head()